In [11]:
#FOLLOWS THE SAME FLOW AS IN appendparticipationdata.ipynb
import pandas as pd
from itertools import groupby
from operator import itemgetter

In [12]:
df = pd.read_csv('../rawdata/user_profiles_participation.csv')

In [13]:
df.columns = ['Years', 'Years Label', 'index', 'url']
df = df[['Years', 'url']]
df.head()

,Years,url
0,NaN,http://nanowrimo.org/participants/nicaless
1,2014 winner,http://nanowrimo.org/participants/nicaless
2,2015 participant,http://nanowrimo.org/participants/nicaless
3,2005 winner,http://nanowrimo.org/participants/rachel-b-moore
4,2006 winner,http://nanowrimo.org/participants/rachel-b-moore


In [14]:
years_new_list = []
for i in df['Years']:
    if isinstance(i, float):
        years_new_list.append("donor")
    else:
        years_new_list.append(i.split(" "))
        

In [15]:
years_new_list

['donor',
 ['2014', 'winner'],
 ['2015', 'participant'],
 ['2005', 'winner'],
 ['2006', 'winner'],
 'donor',
 ['2007', 'winner'],
 'donor',
 ['2008', 'winner'],
 'donor',
 ['2009', 'winner'],
 'donor',
 ['2010', 'winner'],
 'donor',
 ['2011', 'winner'],
 'donor',
 ['2012', 'participant'],
 'donor',
 ['2013', 'participant'],
 'donor',
 ['2014', 'winner'],
 'donor',
 ['2015', 'winner'],
 'donor',
 ['2015', 'participant'],
 ['2004', 'winner'],
 ['2005', 'winner'],
 ['2006', 'participant'],
 'donor',
 ['2007', 'participant'],
 'donor',
 ['2008', 'winner'],
 'donor',
 ['2009', 'participant'],
 ['2010', 'participant'],
 'donor',
 ['2011', 'winner'],
 'donor',
 ['2012', 'winner'],
 'donor',
 ['2013', 'winner'],
 'donor',
 ['2014', 'winner'],
 'donor',
 ['2015', 'winner'],
 ['2012', 'participant'],
 ['2014', 'participant'],
 ['2015', 'participant'],
 ['2012', 'participant'],
 ['2013', 'participant'],
 ['2014', 'participant'],
 ['2015', 'participant'],
 'donor',
 ['2015', 'winner'],
 ['2015', '

In [16]:
df['Years'] = years_new_list

In [17]:
writernames = pd.read_csv('../clean data/user_summary.csv')

In [18]:
writernames = writernames[['Writer Name', 'url']]
len(writernames)

501

In [19]:
new_df = pd.merge(writernames, df, on='url')
new_df = new_df[['Writer Name', 'Years']]
new_df.head()

,Writer Name,Years
0,Nicaless,donor
1,Nicaless,"[2014, winner]"
2,Nicaless,"[2015, participant]"
3,Rachel B. Moore,"[2005, winner]"
4,Rachel B. Moore,"[2006, winner]"


In [20]:
allnames = writernames[['Writer Name']]
allnames.head()

,Writer Name
0,Nicaless
1,Rachel B. Moore
2,abookishbabe
3,alexabexis
4,AllYellowFlowers


In [21]:
groupnames = new_df.groupby('Writer Name')

In [22]:
list_dics = []
test = ["alexabexis"]
#for name in test:
for name in allnames['Writer Name']:
    group = groupnames.get_group(name)
    dic = {}
    donor_years = []
    win_years = []
    part_years = []
    current_year_donor = 0
    current_year_winner = 0
    
    # get all years the writer was a donor/winner/participant excluding 2015
    for i in group.index:
        if group['Years'][i] == 'donor':
            year = group['Years'][i+1][0]
            if year == '2015':
                # indicate if they are donor for 2015
                current_year_donor = 1
                continue
            donor_years.append(group['Years'][i+1][0])
        elif group['Years'][i][1] == "winner":
            year = group['Years'][i][0] 
            if year == '2015':
                # indicate if they are a winner for 2015
                current_year_winner = 1
                continue
            win_years.append(year)
            part_years.append(year)
        else:
            year = group['Years'][i][0] 
            if year == '2015':
                continue
            part_years.append(year)
    
        
    donor_years = map(int, donor_years)
    consec_donor = []
    for k, g in groupby(enumerate(donor_years), lambda (i, x): i-x):
        consec_donor.append(len(map(itemgetter(1), g)))
        
    win_years = map(int, win_years)
    consec_wins = []
    for k, g in groupby(enumerate(win_years), lambda (i, x): i-x):
        consec_wins.append(len(map(itemgetter(1), g)))
    
    part_years = map(int, part_years)
    consec_part = []
    for k, g in groupby(enumerate(part_years), lambda (i, x): i-x):
        consec_part.append(len(map(itemgetter(1), g)))
    
    dic['Writer Name'] = name
    dic['Wins'] = sum(consec_wins)
    dic['Participated'] = sum(consec_part)
    dic['Donations'] = sum(consec_donor)
    if len(consec_donor) < 1:
        dic['Consecutive Donor'] = 0
    else:
        dic['Consecutive Donor'] = max(consec_donor)
    if len(consec_wins) < 1:
        dic['Consecutive Wins'] = 0
    else:
        dic['Consecutive Wins'] = max(consec_wins)
    if len(consec_part) < 1:
        dic['Consecutive Part'] = 0
    else:
        dic['Consecutive Part'] = max(consec_part)

    dic['Donor Years'] = donor_years
    dic['Win Years'] = win_years
    dic['Part Years'] = part_years
    dic['CURRENT WINNER'] = current_year_winner
    dic['Current Donor'] = current_year_donor
    list_dics.append(dic)

In [23]:
final_df = pd.DataFrame(list_dics)
final_df

,CURRENT WINNER,Consecutive Donor,Consecutive Part,Consecutive Wins,Current Donor,Donations,Donor Years,Part Years,Participated,Win Years,Wins,Writer Name
0,0,1,1,1,0,1,[2014],[2014],1,[2014],1,Nicaless
1,1,8,10,7,1,8,"[2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014]","[2005, 2006, 2007, 2008, 2009, 2010, 2011, 201...",10,"[2005, 2006, 2007, 2008, 2009, 2010, 2011, 2014]",8,Rachel B. Moore
2,0,0,0,0,1,0,[],[],0,[],0,abookishbabe
3,1,4,11,4,1,7,"[2007, 2008, 2009, 2011, 2012, 2013, 2014]","[2004, 2005, 2006, 2007, 2008, 2009, 2010, 201...",11,"[2004, 2005, 2008, 2011, 2012, 2013, 2014]",7,alexabexis
4,0,0,1,0,0,0,[],"[2012, 2014]",2,[],0,AllYellowFlowers
5,0,0,3,0,0,0,[],"[2012, 2013, 2014]",3,[],0,animalrza
6,1,0,0,0,1,0,[],[],0,[],0,Cafenuit
7,0,0,0,0,0,0,[],[],0,[],0,Carmelon
8,0,0,1,0,0,0,[],[2012],1,[],0,carozy
9,1,4,4,4,1,4,"[2011, 2012, 2013, 2014]","[2011, 2012, 2013, 2014]",4,"[2011, 2012, 2013, 2014]",4,chrisk0


In [37]:
final_df = final_df[['Writer Name', 'CURRENT WINNER', 'Current Donor', 'Wins', 'Donations', 'Participated', 'Consecutive Donor', 'Consecutive Wins', 'Consecutive Part', 'Part Years', 'Win Years', "Donor Years"]]

In [38]:
#final_df.to_csv("../clean data/user_participation_parsed_no2015.csv")
final_df = final_df[['CURRENT WINNER', 'Current Donor', 'Wins', 'Donations', 'Participated', 'Consecutive Donor', 'Consecutive Wins', 'Consecutive Part', 'Part Years', 'Win Years', "Donor Years"]]

In [39]:
summary = pd.read_csv("../clean data/user_summary_no2015.csv")
summary.columns

Index([u'Writer Name', u'Member Length', u'LifetimeWordCount', u'url', u'Age',
       u'Birthday', u'Favorite books or authors', u'Favorite noveling music',
       u'Hobbies', u'Location', u'Occupation', u'Primary Role',
       u'Sponsorship URL', u'Expected Final Word Count',
       u'Expected Daily Average', u'Num Novels', u'Expected Num Submissions',
       u'Expected Avg Submission', u'Expected Min Submission',
       u'Expected Min Day', u'Expected Max Submission', u'Expected Max Day',
       u'Expected Std Submissions', u'Expected Consec Subs'],
      dtype='object')

In [41]:
summary['CURRENT WINNER'] = final_df['CURRENT WINNER']
summary['Current Donor'] = final_df['Current Donor']
summary['Consecutive Donor'] = final_df['Consecutive Donor']
summary['Consecutive Wins'] = final_df['Consecutive Wins']
summary['Consecutive Part'] = final_df['Consecutive Part']
summary['Part Years'] = final_df['Part Years']
summary['Win Years'] = final_df['Win Years']
summary["Donor Years"] = final_df['Donor Years']

In [42]:
summary.columns

Index([u'Writer Name', u'Member Length', u'LifetimeWordCount', u'url', u'Age',
       u'Birthday', u'Favorite books or authors', u'Favorite noveling music',
       u'Hobbies', u'Location', u'Occupation', u'Primary Role',
       u'Sponsorship URL', u'Expected Final Word Count',
       u'Expected Daily Average', u'Num Novels', u'Expected Num Submissions',
       u'Expected Avg Submission', u'Expected Min Submission',
       u'Expected Min Day', u'Expected Max Submission', u'Expected Max Day',
       u'Expected Std Submissions', u'Expected Consec Subs', u'CURRENT WINNER',
       u'Current Donor', u'Wins', u'Donations', u'Participated',
       u'Consecutive Donor', u'Consecutive Wins', u'Consecutive Part',
       u'Part Years', u'Win Years', u'Donor Years', u'CURRENT WINNER',
       u'Current Donor', u'Wins', u'Donations', u'Participated',
       u'Consecutive Donor', u'Consecutive Wins', u'Consecutive Part',
       u'Part Years', u'Win Years', u'Donor Years'],
      dtype='object')

In [43]:
summary.to_csv("../clean data/user_summary_no2015.csv")

In [45]:
summary.columns

Index([u'Writer Name', u'Member Length', u'LifetimeWordCount', u'url', u'Age',
       u'Birthday', u'Favorite books or authors', u'Favorite noveling music',
       u'Hobbies', u'Location', u'Occupation', u'Primary Role',
       u'Sponsorship URL', u'Expected Final Word Count',
       u'Expected Daily Average', u'Num Novels', u'Expected Num Submissions',
       u'Expected Avg Submission', u'Expected Min Submission',
       u'Expected Min Day', u'Expected Max Submission', u'Expected Max Day',
       u'Expected Std Submissions', u'Expected Consec Subs', u'CURRENT WINNER',
       u'Current Donor', u'Wins', u'Donations', u'Participated',
       u'Consecutive Donor', u'Consecutive Wins', u'Consecutive Part',
       u'Part Years', u'Win Years', u'Donor Years', u'CURRENT WINNER',
       u'Current Donor', u'Wins', u'Donations', u'Participated',
       u'Consecutive Donor', u'Consecutive Wins', u'Consecutive Part',
       u'Part Years', u'Win Years', u'Donor Years'],
      dtype='object')